# Excel Sheet Metadata Extraction - tsDTA Processor

**Purpose**: Extract sheet metadata (names, indices, categories) from tsDTA Excel documents stored in md_file_history table.

**Input**: md_file_history table (filtered for tsDTA Excel documents with READY_FOR_PROCESSING status)

**Output**: md_dta_excel_file_raw table (one row per sheet - metadata only, NO data content)

**Processing**: Scalable parallel processing using mapInPandas for millions of documents

**Filters Applied**:
- Tags contain "tsDTA"
- Active = true
- Current status = "READY_FOR_PROCESSING"
- File extension matches Excel formats (.xls, .xlsx, .xlsm, .xlsb, .xltx, .xltm)

**Optimizations**: 
- mapInPandas for distributed parallel processing across Spark workers
- Z-ORDER by (document_id, sheet_category) for fast queries
- Conditional OPTIMIZE to prevent small file accumulation
- Per-document status tracking for lineage


In [ ]:
# Cell 1: Imports
import os
import json
import base64
import uuid
import pandas as pd
from io import BytesIO
from datetime import datetime
from pyspark.sql import functions as F
from pyspark.sql.functions import lit, current_timestamp
from delta.tables import DeltaTable

from clinical_data_standards_framework.utils import (
    get_excel_sheets_schema,
    
    categorize_sheet_name,
    extract_version_history_metadata,
    save_with_audit
)


In [ ]:
# Cell 2: Get Configuration from Setup Task
globals_dict = json.loads(dbutils.jobs.taskValues.get(taskKey="setup", key="globals"))
pipeline_config = json.loads(dbutils.jobs.taskValues.get(taskKey="setup", key="pipeline_config"))
created_by_principal = dbutils.jobs.taskValues.get(taskKey="setup", key="created_by_principal")
databricks_job_id = dbutils.jobs.taskValues.get(taskKey="setup", key="databricks_job_id")
databricks_job_name = dbutils.jobs.taskValues.get(taskKey="setup", key="databricks_job_name")
databricks_run_id = dbutils.jobs.taskValues.get(taskKey="setup", key="databricks_run_id")

# Extract needed values from globals
catalog = globals_dict['catalog']
schema = globals_dict['bronze_schema']

print(f"Catalog: {catalog}, Schema: {schema}")
print(f"Created by: {created_by_principal}")
print(f"Job ID: {databricks_job_id}, Job Name: {databricks_job_name}, Run ID: {databricks_run_id}")


In [ ]:
# Cell 3: Extract Pipeline-Specific Config
source_config = pipeline_config['xls']['standard']['source']
output_config = pipeline_config['xls']['standard']['output']

source_table = source_config['source_table']
filter_tags = source_config['filter_tags']
filter_active = source_config['filter_active']
filter_status = source_config['filter_status']
excel_sheets_table = output_config['excel_sheets_table']
excel_sheets_table_description = output_config.get('excel_sheets_table_description', '')

# Construct full table names
source_table_full_name = f"{catalog}.{schema}.{source_table}"
excel_sheets_table_full_name = f"{catalog}.{schema}.{excel_sheets_table}"
print(f"Source table: {source_table_full_name}")
print(f"Output table: {excel_sheets_table_full_name}")
print(f"Filter: tags={filter_tags}, active={filter_active}, status={filter_status}")


In [ ]:
# Cell 4: Read Documents - BATCH MODE
# Read from md_file_history table (batch read, not streaming)
# This avoids circular dependencies when updating the same table
df_source = spark.read.format("delta").table(source_table_full_name)

# Filter for tsDTA Excel documents that are ready for processing
# File pattern: *.xls* matches .xls, .xlsx, .xlsm, .xlsb, .xltx, .xltm
# Only process current versions (prevents duplicate processing when files are re-uploaded)
df_filtered = df_source.filter(
    (F.array_contains(F.col("document_tags"), filter_tags[0])) &
    (F.col("active") == filter_active) &
    (F.col("status") == filter_status) &
    (F.col("is_current") == True) &  # ← Only process current version
    (F.col("file_extension").rlike("^\\.(xls|xlsx|xlsm|xlsb|xltx|xltm)$"))
)

# Count documents to process
doc_count = df_filtered.count()
print(f"{'='*80}")
print(f"Documents to process: {doc_count}")
print(f"{'='*80}")

if doc_count == 0:
    print("⚠ No documents to process.")
    print(f"Checking source table {source_table_full_name}...")
    
    total_count = df_source.count()
    print(f"  Total records in source: {total_count}")
    
    tag_count = df_source.filter(F.array_contains(F.col("document_tags"), filter_tags[0])).count()
    print(f"  Records with tag '{filter_tags[0]}': {tag_count}")
    
    active_count = df_source.filter(F.col("active") == filter_active).count()
    print(f"  Records with active={filter_active}: {active_count}")
    
    status_count = df_source.filter(F.col("status") == filter_status).count()
    print(f"  Records with status='{filter_status}': {status_count}")
    
    current_count = df_source.filter(F.col("is_current") == True).count()
    print(f"  Records with is_current=true: {current_count}")
    
    excel_count = df_source.filter(F.col("file_extension").rlike("^\\.(xls|xlsx|xlsm|xlsb|xltx|xltm)$")).count()
    print(f"  Records with Excel extensions: {excel_count}")
    
    # Update status for documents that have the tag but failed filter criteria
    # This helps identify misconfigured uploads (e.g., active=false)
    docs_with_tag_not_active = df_source.filter(
        (F.array_contains(F.col("document_tags"), filter_tags[0])) &
        (F.col("status") == filter_status) &
        (F.col("is_current") == True) &
        (F.col("active") == False)
    ).count()
    
    if docs_with_tag_not_active > 0:
        print(f"\n⚠️  Found {docs_with_tag_not_active} documents with tag '{filter_tags[0]}' but active=false")
        print("   These documents were likely uploaded with incorrect settings.")
        
        # Update these documents with a note explaining why they weren't processed
        spark.sql(f"""
            UPDATE {source_table_full_name}
            SET status = 'XLS_SKIPPED',
                notes = CONCAT(COALESCE(notes, ''), ' | Skipped: active=false, document not eligible for processing'),
                last_updated_ts = current_timestamp()
            WHERE array_contains(document_tags, '{filter_tags[0]}')
              AND status = '{filter_status}'
              AND is_current = true
              AND active = false
        """)
        print(f"   Updated {docs_with_tag_not_active} documents with status='XLS_SKIPPED'")
    
    # Set task value to signal downstream tasks to skip
    dbutils.jobs.taskValues.set(key="documents_found", value="false")
    dbutils.jobs.taskValues.set(key="document_count", value="0")
    
    print("\n⏭️  Setting task value 'documents_found' = false")
    print("   Downstream tasks will be skipped.")
    dbutils.notebook.exit("No documents to process - downstream tasks will be skipped")

# Set task value indicating documents were found
dbutils.jobs.taskValues.set(key="documents_found", value="true")
dbutils.jobs.taskValues.set(key="document_count", value=str(doc_count))
print(f"\n✓ Setting task value 'documents_found' = true")
print(f"  Document count: {doc_count}")


In [ ]:
# Cell 5: Process Excel Files - mapInPandas for Scalable Parallel Processing
# This approach distributes processing across Spark workers for millions of documents

from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType, DoubleType

# Define result schema for mapInPandas output
# This includes both sheet records and processing status per document
# Note: DoubleType used for processing_duration_seconds to match ()
result_schema = StructType([
    StructField("record_type", StringType(), False),  # 'sheet' or 'status' or 'update'
    StructField("document_id", StringType(), False),
    StructField("parent_document_id", StringType(), True),
    StructField("source_file_path", StringType(), True),
    StructField("sheet_name", StringType(), True),
    StructField("sheet_index", IntegerType(), True),
    StructField("sheet_category", StringType(), True),
    StructField("trial_id", StringType(), True),
    StructField("data_stream_type", StringType(), True),
    StructField("data_provider_name", StringType(), True),
    StructField("status", StringType(), True),
    StructField("status_timestamp", TimestampType(), True),
    StructField("error_message", StringType(), True),
    StructField("processing_duration_seconds", DoubleType(), True),
    StructField("processing_metadata", StringType(), True),
    StructField("new_status", StringType(), True),
    StructField("new_status_timestamp", TimestampType(), True)
])

def process_excel_pandas(pdf_iterator):
    """
    Process Excel documents using mapInPandas.
    Each partition is processed in parallel across Spark workers.
    Yields sheet records, status records, and update records for each document.
    
    Parent Document ID Logic:
    - For files extracted from a ZIP: use fh_parent_document_id (the ZIP's document_id)
    - For standalone uploads: fh_parent_document_id is NULL, use the file's document_id as fallback
    """
    for pdf in pdf_iterator:
        results = []
        
        for _, row in pdf.iterrows():
            document_id = row['document_id']
            extracted_path = row['extracted_path']
            content_b64 = row['content_base64']
            # Get parent_document_id from md_file_history
            # For ZIP extracts: this is the ZIP's document_id
            # For standalone uploads: this is None/NaN
            fh_parent_document_id = row.get('fh_parent_document_id')
            
            # Determine the correct parent_document_id:
            # - If file came from ZIP: use the ZIP's document_id (fh_parent_document_id)
            # - If standalone file: use the file's own document_id as the parent
            if pd.notna(fh_parent_document_id) and fh_parent_document_id:
                actual_parent_doc_id = str(fh_parent_document_id)
            else:
                actual_parent_doc_id = document_id
            
            start_time = datetime.now()
            
            try:
                # Decode base64 content
                content_binary = base64.b64decode(content_b64)
                excel_file = BytesIO(content_binary)
                
                # Detect file extension and choose correct engine
                file_ext = os.path.splitext(extracted_path.lower())[1]
                
                if file_ext in ['.xls', '.xlt']:
                    xl = pd.ExcelFile(excel_file, engine='xlrd')
                else:
                    xl = pd.ExcelFile(excel_file, engine='openpyxl')
                
                # Extract Version History metadata once per Excel file
                version_metadata = extract_version_history_metadata(content_binary, file_ext)
                trial_id = version_metadata.get("trial_id", "UNKNOWN")
                data_stream_type = version_metadata.get("data_stream_type", "UNKNOWN")
                data_provider_name = version_metadata.get("data_provider_name", "UNKNOWN")
                
                # Extract each sheet metadata with category
                for sheet_idx, sheet_name in enumerate(xl.sheet_names):
                    sheet_category = categorize_sheet_name(sheet_name)
                    
                    results.append({
                        'record_type': 'sheet',
                        'document_id': str(uuid.uuid4()),  # Unique sheet ID
                        'parent_document_id': actual_parent_doc_id,  # Use correct parent (ZIP or self)
                        'source_file_path': extracted_path,
                        'sheet_name': sheet_name,
                        'sheet_index': sheet_idx,
                        'sheet_category': sheet_category,
                        'trial_id': trial_id,
                        'data_stream_type': data_stream_type,
                        'data_provider_name': data_provider_name,
                        'status': None,
                        'status_timestamp': None,
                        'error_message': None,
                        'processing_duration_seconds': None,
                        'processing_metadata': None,
                        'new_status': None,
                        'new_status_timestamp': None
                    })
                
                # Calculate duration
                end_time = datetime.now()
                duration = (end_time - start_time).total_seconds()
                
                # Status record: COMPLETED
                results.append({
                    'record_type': 'status',
                    'document_id': document_id,
                    'parent_document_id': None,
                    'source_file_path': None,
                    'sheet_name': None,
                    'sheet_index': None,
                    'sheet_category': None,
                    'trial_id': None,
                    'data_stream_type': None,
                    'data_provider_name': None,
                    'status': 'EXCEL_EXTRACTION_COMPLETED',
                    'status_timestamp': end_time,
                    'error_message': None,
                    'processing_duration_seconds': duration,
                    'processing_metadata': json.dumps({
                        'sheets_extracted': len(xl.sheet_names),
                        'source_path': extracted_path
                    }),
                    'new_status': None,
                    'new_status_timestamp': None
                })
                
                # Update record for manifest
                results.append({
                    'record_type': 'update',
                    'document_id': document_id,
                    'parent_document_id': None,
                    'source_file_path': None,
                    'sheet_name': None,
                    'sheet_index': None,
                    'sheet_category': None,
                    'trial_id': None,
                    'data_stream_type': None,
                    'data_provider_name': None,
                    'status': None,
                    'status_timestamp': None,
                    'error_message': None,
                    'processing_duration_seconds': None,
                    'processing_metadata': None,
                    'new_status': 'EXCEL_EXTRACTION_COMPLETED',
                    'new_status_timestamp': end_time
                })
                
            except Exception as e:
                # Error handling - create error records
                error_time = datetime.now()
                duration = (error_time - start_time).total_seconds()
                
                # Status record: ERROR
                results.append({
                    'record_type': 'status',
                    'document_id': document_id,
                    'parent_document_id': None,
                    'source_file_path': None,
                    'sheet_name': None,
                    'sheet_index': None,
                    'sheet_category': None,
                    'trial_id': None,
                    'data_stream_type': None,
                    'data_provider_name': None,
                    'status': 'EXCEL_EXTRACTION_ERROR',
                    'status_timestamp': error_time,
                    'error_message': str(e)[:1000],  # Truncate long errors
                    'processing_duration_seconds': duration,
                    'processing_metadata': json.dumps({'source_path': extracted_path}),
                    'new_status': None,
                    'new_status_timestamp': None
                })
                
                # Update record for manifest
                results.append({
                    'record_type': 'update',
                    'document_id': document_id,
                    'parent_document_id': None,
                    'source_file_path': None,
                    'sheet_name': None,
                    'sheet_index': None,
                    'sheet_category': None,
                    'trial_id': None,
                    'data_stream_type': None,
                    'data_provider_name': None,
                    'status': None,
                    'status_timestamp': None,
                    'error_message': None,
                    'processing_duration_seconds': None,
                    'processing_metadata': None,
                    'new_status': 'EXCEL_EXTRACTION_ERROR',
                    'new_status_timestamp': error_time
                })
        
        # Yield results as DataFrame
        if results:
            yield pd.DataFrame(results)
        else:
            # Yield empty DataFrame with correct schema
            yield pd.DataFrame(columns=[f.name for f in result_schema.fields])

print(f"✓ mapInPandas processing function defined")
print(f"  - Parallel processing across Spark workers")
print(f"  - Per-document status tracking for lineage")
print(f"  - Error handling with detailed messages")


In [ ]:
# Cell 6: Execute mapInPandas and Write Results
print(f"{'='*80}")
print(f"Starting parallel processing with mapInPandas...")
print(f"Documents to process: {doc_count}")
print(f"{'='*80}")

# Repartition for optimal parallelism (1 partition per ~10 docs, min 1, max 200)
num_partitions = min(max(1, doc_count // 10), 200)
# Include parent_document_id from md_file_history to correctly link sheets to ZIP parent
# Rename to fh_parent_document_id to distinguish from the sheet's parent_document_id
df_repartitioned = df_filtered.select(
    'document_id',
    F.col('parent_document_id').alias('fh_parent_document_id'),  # From file history (ZIP's doc_id or NULL)
    'extracted_path',
    'content_base64'
).repartition(num_partitions)

print(f"Using {num_partitions} partitions for parallel processing")
print(f"  Including fh_parent_document_id for ZIP file lineage tracking")

# Execute mapInPandas - this runs in parallel across Spark workers
df_results = df_repartitioned.mapInPandas(process_excel_pandas, result_schema)

# Note: cache() not supported on serverless compute - Spark will recompute as needed

# Split results by record_type
df_sheets = df_results.filter(F.col("record_type") == "sheet").select(
    "document_id", "parent_document_id", "source_file_path",
    "sheet_name", "sheet_index", "sheet_category",
    "trial_id", "data_stream_type", "data_provider_name"
)

df_status = df_results.filter(F.col("record_type") == "status").select(
    "document_id", "status", "status_timestamp",
    "error_message", "processing_duration_seconds", "processing_metadata"
).withColumn("retry_count", lit(0))

df_updates = df_results.filter(F.col("record_type") == "update").select(
    "document_id", "new_status", "new_status_timestamp"
)

# Count results
sheet_count = df_sheets.count()
success_count = df_status.filter(F.col("status") == "EXCEL_EXTRACTION_COMPLETED").count()
error_count = df_status.filter(F.col("status") == "EXCEL_EXTRACTION_ERROR").count()

print(f"\n{'='*80}")
print(f"Processing Results:")
print(f"  Sheets extracted: {sheet_count}")
print(f"  Documents SUCCESS: {success_count}")
print(f"  Documents ERROR: {error_count}")
print(f"{'='*80}")

# Write sheet records
if sheet_count > 0:
    print(f'\nSaving {sheet_count} sheets to {excel_sheets_table_full_name}...')
    save_with_audit(
        df=df_sheets,
        table_name=excel_sheets_table_full_name,
        created_by_principal=created_by_principal,
        databricks_job_id=databricks_job_id,
        databricks_job_name=databricks_job_name,
        databricks_run_id=databricks_run_id,
        mode='append'
    )
    print(f'✓ Saved {sheet_count} sheets with audit columns')

# Update md_file_history table with processing status via MERGE
# Status is now consolidated in md_file_history - no separate status table needed
update_count = df_updates.count()
if update_count > 0:
    print(f'\nUpdating {update_count} documents in manifest...')
    delta_table = DeltaTable.forName(spark, source_table_full_name)
    
    delta_table.alias("target").merge(
        df_updates.alias("updates"),
        "target.document_id = updates.document_id"
    ).whenMatchedUpdate(set={
        "status": "updates.new_status",
        "status_timestamp": "updates.new_status_timestamp",
        "last_updated_by_principal": lit(created_by_principal),
        "last_updated_ts": current_timestamp(),
        "databricks_job_id": lit(databricks_job_id),
        "databricks_job_name": lit(databricks_job_name),
        "databricks_run_id": lit(databricks_run_id)
    }).execute()
    
    print(f'✓ Updated {update_count} documents')

# Set task values for downstream tasks
dbutils.jobs.taskValues.set(key="excel_extraction_status", value="SUCCESS" if success_count > 0 else "FAILED")
dbutils.jobs.taskValues.set(key="excel_sheets_count", value=str(sheet_count))
dbutils.jobs.taskValues.set(key="excel_success_count", value=str(success_count))
dbutils.jobs.taskValues.set(key="excel_error_count", value=str(error_count))

# Set parent_document_id for downstream tasks (required for test concepts, codelists, etc.)
if sheet_count > 0:
    first_parent_doc = df_sheets.select("parent_document_id").first()
    if first_parent_doc and first_parent_doc['parent_document_id']:
        dbutils.jobs.taskValues.set(key="parent_document_id", value=first_parent_doc['parent_document_id'])
        print(f"  Set parent_document_id: {first_parent_doc['parent_document_id']}")
    else:
        print("  ⚠ No parent_document_id found in sheets")
else:
    print("  ⚠ No sheets processed - parent_document_id not set")

print(f"\n{'='*80}")
print(f"✓ Parallel processing completed!")
print(f"{'='*80}")


In [ ]:
# Cell 7: Conditional OPTIMIZE - md_dta_excel_file_raw table
# Only run OPTIMIZE when conditions are met to save compute resources

# Get table stats
table_stats = spark.sql(f"DESCRIBE DETAIL {excel_sheets_table_full_name}").first()
num_files = table_stats.numFiles
total_size_gb = table_stats.sizeInBytes / (1024**3)
avg_file_size_mb = (total_size_gb * 1024) / num_files if num_files > 0 else 0

print(f"\n{'='*80}")
print(f"TABLE STATS: {excel_sheets_table_full_name}")
print(f"{'='*80}")
print(f"Sheets processed: {doc_count}")
print(f"Total files: {num_files}")
print(f"Total size: {total_size_gb:.2f} GB")
print(f"Avg file size: {avg_file_size_mb:.2f} MB")

# Determine if OPTIMIZE is needed
should_optimize = False
reason = ""

if doc_count > 100:  # Condition 1: Large batch
    should_optimize = True
    reason = f"Large batch processed ({doc_count} documents)"
elif num_files > 200 and avg_file_size_mb < 100:  # Condition 2: Small file problem
    should_optimize = True
    reason = f"Small file problem ({num_files} files, avg {avg_file_size_mb:.2f}MB)"
elif num_files > 500:  # Condition 3: Too many files
    should_optimize = True
    reason = f"Too many files ({num_files})"

if should_optimize:
    print(f"\n🔧 OPTIMIZE TRIGGERED: {reason}")
    print(f"{'='*80}")
    
    start_time = datetime.now()
    
    # Run OPTIMIZE with Z-ORDER
    spark.sql(f"""
        OPTIMIZE {excel_sheets_table_full_name}
        ZORDER BY (document_id, sheet_category)
    """)
    
    # Get stats after optimization
    table_stats_after = spark.sql(f"DESCRIBE DETAIL {excel_sheets_table_full_name}").first()
    num_files_after = table_stats_after.numFiles
    
    duration = (datetime.now() - start_time).total_seconds()
    
    print(f"✓ OPTIMIZE completed in {duration:.1f} seconds")
    print(f"  Files: {num_files} → {num_files_after} ({num_files - num_files_after} merged)")
    print(f"{'='*80}")
else:
    print(f"\n⏭️  OPTIMIZE SKIPPED: Conditions not met")
    print(f"  - Docs processed: {doc_count} (threshold: >100)")
    print(f"  - Files: {num_files} (threshold: >200 with small files OR >500 total)")
    print(f"  - Avg file size: {avg_file_size_mb:.2f}MB (threshold: <100MB)")
    print(f"{'='*80}")
